In [1]:
import pickle
import torch
from transformer_lens import HookedTransformerConfig, HookedTransformer
from transformer_lens import HookedTransformer
from circuits_benchmark.utils.get_cases import get_cases

task = get_cases(indices=['3'])[0]
task_idx = task.get_index()

In [2]:
from circuits_benchmark.utils.iit.wandb_loader import load_model_from_wandb
import pickle

load_model_from_wandb(case_index=task_idx, weights="100")
cfg = pickle.load(open(f"./results/ll_models/{task_idx}/ll_model_cfg_100.pkl", "rb"))
model = HookedTransformer(cfg)
model.load_state_dict(torch.load(f"./results/ll_models/{task_idx}/ll_model_100.pth", map_location='cpu'))

<All keys matched successfully>

In [3]:
# load high level model
from circuits_benchmark.utils.iit import make_iit_hl_model
import circuits_benchmark.utils.iit.correspondence as correspondence
import iit.model_pairs as mp

def make_model_pair(benchmark_case):
    hl_model = benchmark_case.build_transformer_lens_model()
    hl_model = make_iit_hl_model(hl_model, eval_mode=True)
    tracr_output = benchmark_case.get_tracr_output()
    hl_ll_corr = correspondence.TracrCorrespondence.from_output(
            case=benchmark_case, tracr_output=tracr_output
        )
    model_pair = mp.StrictIITModelPair(hl_model, model, hl_ll_corr)
    return model_pair

model_pair = make_model_pair(task)

Moving model to device:  cpu
{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': HookPoint(), 'blocks.1.attn.hook_

In [4]:
from circuits_benchmark.utils.iit.dataset import get_unique_data
max_len = 256
unique_test_data = get_unique_data(task, max_len=max_len)
def collate_fn(batch):
    encoded_x = model_pair.hl_model.map_tracr_input_to_tl_input(list(zip(*batch))[0])
    return encoded_x

loader = torch.utils.data.DataLoader(unique_test_data, batch_size=256, shuffle=False, drop_last=False, collate_fn=collate_fn)

In [5]:
import utils.node_stats as node_stats
cache_dict = node_stats.get_node_stats(model_pair, loader)
node_norms = node_stats.node_stats_to_df(cache_dict)


/Users/cybershiptrooper/src/interpretability/MATS/circuits-benchmark/notebooks/utils/node_stats.py:87: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  node_norms = pd.concat(


In [6]:
# delete in_circuit column from node_norms
# node_norms = node_norms.drop(columns=['in_circuit'])
node_norms

,name,in_circuit,norm_cache,norm_std
0,"blocks.0.attn.hook_result, head 0",False,0.406032,0.119806
1,"blocks.0.attn.hook_result, head 1",False,0.306258,0.102561
2,"blocks.0.attn.hook_result, head 2",False,0.364599,0.022109
3,"blocks.0.attn.hook_result, head 3",False,0.379827,0.038986
6,"blocks.1.attn.hook_result, head 2",False,0.540073,0.156250
7,"blocks.1.attn.hook_result, head 3",False,0.555689,0.145491
8,blocks.1.mlp.hook_post,False,2.221375,0.633349
4,blocks.0.mlp.hook_post,True,2.038637,0.164101
5,"blocks.1.attn.hook_result, head :2",True,1.066467,0.254025


In [7]:
import circuits_benchmark.commands.evaluation.iit.iit_eval as eval_node_effect

# model_pair = make_model_pair(task)
args = eval_node_effect.setup_args_parser(None, True)
max_len = 50
args.max_len = max_len
model_pair = make_model_pair(task)
node_effects, eval_metrics = eval_node_effect.get_node_effects(case=task, model_pair=model_pair, args=args, use_mean_cache=False)

Moving model to device:  cpu
{'hook_embed': HookPoint(), 'hook_pos_embed': HookPoint(), 'blocks.0.attn.hook_k': HookPoint(), 'blocks.0.attn.hook_q': HookPoint(), 'blocks.0.attn.hook_v': HookPoint(), 'blocks.0.attn.hook_z': HookPoint(), 'blocks.0.attn.hook_attn_scores': HookPoint(), 'blocks.0.attn.hook_pattern': HookPoint(), 'blocks.0.attn.hook_result': HookPoint(), 'blocks.0.mlp.hook_pre': HookPoint(), 'blocks.0.mlp.hook_post': HookPoint(), 'blocks.0.hook_attn_in': HookPoint(), 'blocks.0.hook_q_input': HookPoint(), 'blocks.0.hook_k_input': HookPoint(), 'blocks.0.hook_v_input': HookPoint(), 'blocks.0.hook_mlp_in': HookPoint(), 'blocks.0.hook_attn_out': HookPoint(), 'blocks.0.hook_mlp_out': HookPoint(), 'blocks.0.hook_resid_pre': HookPoint(), 'blocks.0.hook_resid_mid': HookPoint(), 'blocks.0.hook_resid_post': HookPoint(), 'blocks.1.attn.hook_k': HookPoint(), 'blocks.1.attn.hook_q': HookPoint(), 'blocks.1.attn.hook_v': HookPoint(), 'blocks.1.attn.hook_z': HookPoint(), 'blocks.1.attn.hook_

100%|██████████| 4/4 [00:00<00:00, 54.05it/s]


In [8]:
node_effects

,node,status,resample_ablate_effect,zero_ablate_effect
0,"blocks.0.attn.hook_result, head 0",not_in_circuit,0.964815,0.414258
1,"blocks.0.attn.hook_result, head 1",not_in_circuit,0.734521,0.277148
2,"blocks.0.attn.hook_result, head 2",not_in_circuit,0.117473,0.004102
3,"blocks.0.attn.hook_result, head 3",not_in_circuit,0.542011,0.136719
4,"blocks.1.attn.hook_result, head 2",not_in_circuit,0.428004,0.102148
5,"blocks.1.attn.hook_result, head 3",not_in_circuit,0.055523,0.265820
6,blocks.1.mlp.hook_post,not_in_circuit,0.749058,0.619336
7,"blocks.1.attn.hook_result, head :2",in_circuit,0.684763,0.341211
8,blocks.0.mlp.hook_post,in_circuit,0.887523,0.825977


In [9]:
import pandas as pd
combined_df = pd.merge(node_effects, node_norms, left_on="node", right_on="name", how="inner")
combined_df.drop(columns=["name", "in_circuit"], inplace=True)
combined_df

,node,status,resample_ablate_effect,zero_ablate_effect,norm_cache,norm_std
0,"blocks.0.attn.hook_result, head 0",not_in_circuit,0.964815,0.414258,0.406032,0.119806
1,"blocks.0.attn.hook_result, head 1",not_in_circuit,0.734521,0.277148,0.306258,0.102561
2,"blocks.0.attn.hook_result, head 2",not_in_circuit,0.117473,0.004102,0.364599,0.022109
3,"blocks.0.attn.hook_result, head 3",not_in_circuit,0.542011,0.136719,0.379827,0.038986
4,"blocks.1.attn.hook_result, head 2",not_in_circuit,0.428004,0.102148,0.540073,0.156250
5,"blocks.1.attn.hook_result, head 3",not_in_circuit,0.055523,0.265820,0.555689,0.145491
6,blocks.1.mlp.hook_post,not_in_circuit,0.749058,0.619336,2.221375,0.633349
7,"blocks.1.attn.hook_result, head :2",in_circuit,0.684763,0.341211,1.066467,0.254025
8,blocks.0.mlp.hook_post,in_circuit,0.887523,0.825977,2.038637,0.164101


In [11]:
# combined_df.drop(columns=["in_circuit"], inplace=True)
combined_df.drop(columns=["status"], inplace=True)
combined_df

,node,resample_ablate_effect,zero_ablate_effect,norm_cache,norm_std
0,"blocks.0.attn.hook_result, head 0",0.964815,0.414258,0.406032,0.119806
1,"blocks.0.attn.hook_result, head 1",0.734521,0.277148,0.306258,0.102561
2,"blocks.0.attn.hook_result, head 2",0.117473,0.004102,0.364599,0.022109
3,"blocks.0.attn.hook_result, head 3",0.542011,0.136719,0.379827,0.038986
4,"blocks.1.attn.hook_result, head 2",0.428004,0.102148,0.540073,0.156250
5,"blocks.1.attn.hook_result, head 3",0.055523,0.265820,0.555689,0.145491
6,blocks.1.mlp.hook_post,0.749058,0.619336,2.221375,0.633349
7,"blocks.1.attn.hook_result, head :2",0.684763,0.341211,1.066467,0.254025
8,blocks.0.mlp.hook_post,0.887523,0.825977,2.038637,0.164101
